<center><a href="https://www.together.ai" ><img src="https://raw.githubusercontent.com/clam004/together-examples/main/files/togetherlogo.jpg" align="center"/></a></center>

[together.ai]("https://www.together.ai") allows you to train, finetune and deploy large language models on fast compute using 5 lines of code from your laptop, without dealing with anything having the words *cuda* in it. After you adapt these open source models for your tasks/use cases, you can download the entire model for yourself and/or serve the model on our fast inference engines. see our [docs](https://docs.together.ai/docs/fine-tuning-python) or follow along below.

This python notebook is an example before and after demonstration/tutorial of using Together's finetune API to adapt a base model to a domain specific dataset in law. The dataset consists of legal related questions and answers.

In [ ]:
# First lets load some tools to help us manage out dataset and
!pip install datasets
!pip install --upgrade together

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 898.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.5 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.13
    Uninstalling pydantic-1.10.13:
      Successfully uninstalled pydantic-1.10.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the foll

In the "xxxx" placeholder below, paste your Together API Key. You can get your Together API Key by signing up at [together.ai]("https://www.together.ai"), then going to the profile icon > Settings > API Keys.

<center><img src="https://raw.githubusercontent.com/clam004/together-examples/main/files/SETTINGS.png" height=350 width=250></center>

<center><img src="https://raw.githubusercontent.com/togethercomputer/examples/main/sample_images/UserSettings.png" height=350 width=250></center>


<center><img src="https://raw.githubusercontent.com/clam004/together-examples/main/files/API_KEY.png" height=250 width=350></center>

First lets import our python packages and check to make sure our API Key is a 64 character alphanumeric string. Also make sure you are using the latest version of together API python library.


In [ ]:
WANDB_API_KEY = None # replace None with your weights and Biases API key (optional)
TOGETHER_API_KEY = "xxxx" # replace "xxxx" with your together API key (needed but easy)

In [ ]:
import os
import random

from datasets import load_dataset

import together

# check to make sure you have the right key, ie: xxxx.....c04c
print(f"using TOGETHER_API_KEY ending in {TOGETHER_API_KEY[-4:]}")
print("together.VERSION", together.VERSION)

together.api_key = TOGETHER_API_KEY

using TOGETHER_API_KEY ending in c04c
together.VERSION 0.2.10


Go to https://docs.together.ai/docs/fine-tuning-models to see a full list of our current base models available for finetuning

In [ ]:
# lets use our base model to see how it works before we finetune it

base_model_name = "togethercomputer/Llama-2-7B-32K-Instruct"
#base_model_name = "togethercomputer/llama-2-7b-chat"

See [fine-tuning-task-specific-sequences](https://docs.together.ai/docs/fine-tuning-task-specific-sequences) for a full description of how to template/format/structure your finetuning data for your use case.

The TL;DR is that you can teach your model how to follow a new set of special symbols and grammar, or you can build on top of the patte you can give your model the background knowledge, context or identity in the system prompt `<s>[INST] <<SYS>> {system_prompt} <</SYS>>` and then train it to be consistent with the system prompt instructions as it does multi-turn chat by concatenating repeated ` {user_msg} [/INST] {model_answer} </s>` utterance pairs to the right end of a growing sequence of dialog history. The interface you build will have to use these special sequences like `</s>` and `[/INST]` to extract out the model's responses and input back into the model the growing history of conversational turns. Lets first see how the the model behaves with one method of formatting text, the Llama-2 style.

In [ ]:
test_chat_prompt = "<s>[INST] <<SYS>> you are a helpful legal assistant <</SYS>> Analyze and explain the legal reasoning behind the judgment in the given case.\n\nCentral Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly & Anr., 1986 AIR 1571, 1986 SCR (2) 278 [/INST]"
test_chat_prompt

'<s>[INST] <<SYS>> you are a helpful legal assistant <</SYS>> Analyze and explain the legal reasoning behind the judgment in the given case.\n\nCentral Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly & Anr., 1986 AIR 1571, 1986 SCR (2) 278 [/INST]'

In [ ]:
print(base_model_name)

output = together.Complete.create(
  prompt = test_chat_prompt,
  model = base_model_name,
  max_tokens = 256,
  temperature = 0.6,
  top_k = 90,
  top_p = 0.8,
  repetition_penalty = 1.1,
  stop = ['</s>', '[/INST]']
)

# print generated text
print(output['prompt'][0]+" -> "+output['output']['choices'][0]['text'])

togethercomputer/Llama-2-7B-32K-Instruct
<s>[INST] <<SYS>> you are a helpful legal assistant <</SYS>> Analyze and explain the legal reasoning behind the judgment in the given case.

Central Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly & Anr., 1986 AIR 1571, 1986 SCR (2) 278 [/INST] -> 

The judgment in Central Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly & Anr., 1986 AIR 1571, 1986 SCR (2) 278 was based on the principle of "piercing the corporate veil" to hold the shareholders personally liable for the debts of the company. The court found that the shareholders had used the corporation as a mere instrumentality for their personal gain and had not acted in good faith or with regard to the interests of the corporation.

The court held that the shareholders were liable for the debts of the company because they had been involved in fraudulent activities and had misappropriated the assets of the corporation. The court also noted that the shareholders had no

Lets take a look at structure of the data we will be using


`
<s>[INST] <<SYS>> you are a helpful legal assistant <</SYS>> Analyze and explain the legal reasoning behind the judgment in the given case.:Central Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly & Anr., 1986 AIR 1571, 1986 SCR (2) 278
[/INST]
The Supreme Court in this case applied a broad interpretation of the term 'State' under Article 12 of the Constitution. The court reasoned that a government company undertaking public functions qualifies as 'State' based on factors like government control, public importance of activities etc. This interpretation was based on previous decisions that have defined 'State' under Article 12 broadly to include various agencies and instrumentalities beyond just statutory bodies. The court also applied the principle that unreasonable and arbitrary contractual terms can be struck down under Article 14 of the Constitution. The court found that Rule 9(i) of the service rules, which allowed for termination of service without reason, conferred unfettered power to terminate employment without hearing. This was deemed arbitrary and violative of principles of natural justice and right to equality under Article 14. Furthermore, the court held that the right to life and livelihood under Article 21 is affected by arbitrary termination of employment. The court reasoned that the right to livelihood is an integral part of the right to life, and any arbitrary action that affects a person's livelihood would be a violation of Article 21.In conclusion, the court's legal reasoning was based on a broad interpretation of the term 'State', the application of the principle of equality and natural justice under Article 14, and the protection of the right to life and livelihood under Article 21. </s>
`


In [ ]:
# lets load and take a look at two samples from our 4394 sample dataset
legal_dataset = load_dataset("nisaar/LLAMA2_Legal_Dataset_4.4k_Instructions")

print(legal_dataset)
print("-"*50)
print(legal_dataset['train'][0])
print("-"*50)
print(legal_dataset['train'][1])

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'prompt', 'text'],
        num_rows: 4394
    })
})
--------------------------------------------------
{'instruction': 'Analyze and explain the legal reasoning behind the judgment in the given case.', 'input': 'Central Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly & Anr., 1986 AIR 1571, 1986 SCR (2) 278', 'output': "The Supreme Court in this case applied a broad interpretation of the term 'State' under Article 12 of the Constitution. The court reasoned that a government company undertaking public functions qualifies as 'State' based on factors like government control, public importance of activities etc. This interpretation was based on previous decisions that have defined 'State' under Article 12 broadly to include various agencies and instrumentalities beyond just statutory bodies. The court also applied the principle that unreasonable and arbitrary contractual terms can be struck 

In [ ]:
def format_to_llama2_chat(system_prompt, user_model_chat_list):

    """ this function follows from
    https://docs.together.ai/docs/fine-tuning-task-specific-sequences

    It converts this legal dataset into the Llama-2 prompting structure

    Args:
      system_prompt (str): instructions from you the developer to the AI
      user_model_chat_list (List[Tuple[str,str]]): a list of tuples,
        where each tuple is a pair or exchange of string utterances, the first by the user,
        the second by the AI. The earlier exchanges are on the left, meaning time
        runs left to right.
    Returns:
      growing_prompt (str): the concatenated sequence starting with system_prompt and
        alternating utterances between the user and AI with the last AI utternance on the right.
    """

    growing_prompt = f"""<s>[INST] <<SYS>> {system_prompt} <</SYS>>"""

    for user_msg, model_answer in user_model_chat_list:
        growing_prompt += f""" {user_msg} [/INST] {model_answer} </s>"""

    return growing_prompt

format_to_llama2_chat(
    "You are a good robot",
    [("hi robot", "hello human"),("are you good?", "yes im good"),("are you bad?", "no, im good")]
)

'<s>[INST] <<SYS>> You are a good robot <</SYS>> hi robot [/INST] hello human </s> are you good? [/INST] yes im good </s> are you bad? [/INST] no, im good </s>'

In [ ]:
data_list = []

for sample in legal_dataset['train']:

    instruction_input_separator = random.choice([":", ": ", "\n", "\n\n", " "])
    input = sample['input'] if sample['input'] is not None else ""
    instruction = sample['instruction'] if sample['instruction'] is not None else ""

    training_sequence = format_to_llama2_chat(
        "you are a helpful legal assistant",
        [(instruction+instruction_input_separator+input,sample['output'])]
    )

    data_list.append({
        "text":training_sequence
    })

print(len(data_list))
print(data_list[0])

4394
{'text': "<s>[INST] <<SYS>> you are a helpful legal assistant <</SYS>> Analyze and explain the legal reasoning behind the judgment in the given case.: Central Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly & Anr., 1986 AIR 1571, 1986 SCR (2) 278 [/INST] The Supreme Court in this case applied a broad interpretation of the term 'State' under Article 12 of the Constitution. The court reasoned that a government company undertaking public functions qualifies as 'State' based on factors like government control, public importance of activities etc. This interpretation was based on previous decisions that have defined 'State' under Article 12 broadly to include various agencies and instrumentalities beyond just statutory bodies. The court also applied the principle that unreasonable and arbitrary contractual terms can be struck down under Article 14 of the Constitution. The court found that Rule 9(i) of the service rules, which allowed for termination of service without rea

In [ ]:
# save the reformatted dataset locally
together.Files.save_jsonl(data_list, "legal_dataset.jsonl")

Wrote 4394 records to legal_dataset.jsonl


In [ ]:
# check your data with your base model prompting type before uploading
resp = together.Files.check(file="legal_dataset.jsonl")
print(resp)

{'is_check_passed': True, 'model_special_tokens': 'we are not yet checking end of sentence tokens for this model', 'file_present': 'File found', 'file_size': 'File size 0.007 GB', 'num_samples': 4394}


In [ ]:
resp = together.Files.list()
print(resp)

{'data': [{'filename': 'output.jsonl', 'bytes': 2578, 'created_at': 1690957221, 'id': 'file-3b60de18-37e2-49ba-85e1-b1df893d7eaa', 'purpose': 'fine-tune', 'object': 'file', 'LineCount': 0, 'Processed': True}, {'filename': 'jokes.jsonl', 'bytes': 40805, 'created_at': 1691710036, 'id': 'file-960be810-4d33-449a-885a-9f69bd8fd0e2', 'purpose': 'fine-tune', 'object': 'file', 'LineCount': 0, 'Processed': True}, {'filename': 'bad.jsonl', 'bytes': 158, 'created_at': 1692129949, 'id': 'file-d41d422d-97f8-4edc-89be-54de661db335', 'purpose': 'fine-tune', 'object': 'file', 'LineCount': 0, 'Processed': True}, {'filename': 'bad.jsonl', 'bytes': 179, 'created_at': 1692130538, 'id': 'file-e633bd00-6c65-4196-9228-d2467b420424', 'purpose': 'fine-tune', 'object': 'file', 'LineCount': 0, 'Processed': True}, {'filename': 'bad.jsonl', 'bytes': 219, 'created_at': 1692132918, 'id': 'file-7c7f552d-8c72-41ed-8061-0a3f07d803fe', 'purpose': 'fine-tune', 'object': 'file', 'LineCount': 0, 'Processed': True}, {'filen

In [ ]:
# upload your dataset file to together and save the file-id, youll need it to start your finetuning run
file_resp = together.Files.upload(file="legal_dataset.jsonl")
file_id = file_resp["id"]
print(file_resp)

Uploading legal_dataset.jsonl: 100%|██████████| 6.96M/6.96M [00:01<00:00, 4.61MB/s]

{'filename': 'legal_dataset.jsonl', 'id': 'file-6e235eab-5e2a-44c7-b4f6-b4f336da9de0', 'object': 'file', 'report_dict': {'is_check_passed': True, 'model_special_tokens': 'we are not yet checking end of sentence tokens for this model', 'file_present': 'File found', 'file_size': 'File size 0.007 GB', 'num_samples': 4394}}


Expected output: ```{'filename': 'legal_dataset.jsonl', 'id': 'file-69649a68-6a36-41ad-8420-1750e99c26a7', 'object': 'file', 'report_dict': {'is_check_passed': True, 'model_special_tokens': 'we are not yet checking end of sentence tokens for this model', 'file_present': 'File found', 'file_size': 'File siz```

### Finetuning the base model on the new dataset to create the new finetuned model

depending on the size of the model and dataset this could take a few minutes to hours

In [ ]:
# Submit your finetune job
ft_resp = together.Finetune.create(
  training_file = file_id ,
  model = base_model_name,
  n_epochs = 2,
  batch_size = 4,
  n_checkpoints = 1,
  learning_rate = 5e-5,
  wandb_api_key = WANDB_API_KEY,
  #estimate_price = True,
  suffix = 'law',
)

fine_tune_id = ft_resp['id']
print(ft_resp)

{'training_file': 'file-6e235eab-5e2a-44c7-b4f6-b4f336da9de0', 'validation_file': '', 'model_output_name': 'carson/Llama-2-7B-32K-Instruct-law-2023-12-28-18-38-35', 'model_output_path': 's3://together-dev/finetune/64c4302a5cb247a0c80a3ddb/carson/Llama-2-7B-32K-Instruct-law-2023-12-28-18-38-35/ft-49c89a70-6a65-40c1-bc85-b60c3d2a59f8', 'Suffix': 'law', 'model': 'togethercomputer/Llama-2-7B-32K-Instruct', 'n_epochs': 2, 'n_checkpoints': 1, 'batch_size': 4, 'learning_rate': 5e-05, 'user_id': '64c4302a5cb247a0c80a3ddb', 'lora': False, 'lora_r': 8, 'lora_alpha': 8, 'lora_dropout': 0, 'staring_epoch': 0, 'training_offset': 0, 'checkspoint_path': '', 'random_seed': '', 'created_at': '2023-12-28T18:38:35.492Z', 'updated_at': '2023-12-28T18:38:35.492Z', 'status': 'pending', 'owner_address': '0xef5286fc0a1ac5bc4d4221cf3d51f1d97c45eaf7', 'id': 'ft-49c89a70-6a65-40c1-bc85-b60c3d2a59f8', 'job_id': '', 'token_count': 0, 'param_count': 0, 'total_price': 0, 'epochs_completed': 0, 'events': [{'object': 

Expected output:
```{'training_file': 'file-69649a68-6a36-41ad-8420-1750e99c26a7', 'model_output_name': 'carson/llama-2-7b-chat-law-2023-09-22-20-37-12', 'model_output_path': 's3://together-dev/finetune/64c4302a5cb247a0c80a3ddb/carson/llama-2-7b-chat-law-2023-09-22-20-37-12/ft-2aaecf7b-ff6f-4341-813a-45d84ae2b1bf', 'Suffix': 'law', 'model': 'togethercomputer/llama-2-7b-chat', 'n_epochs': 2, 'n_checkpoints': 1, 'batch_size': 4, 'learning_rate': 0.0001, 'user_id': '64c4302a5cb247a0c80a3ddb', 'staring_epoch': 0, 'training_offset': 0, 'checkspoint_path': '', 'random_seed': '', 'created_at': '2023-09-22T20:37:12.007Z', 'updated_at': '2023-09-22T20:37:12.007Z', 'status': 'pending', 'owner_address': '0xef5286fc0a1ac5bc4d4221cf3d51f1d97c45eaf7', 'id': 'ft-2aaecf7b-ff6f-4341-813a-45d84ae2b1bf', 'job_id': '', 'token_count': 0, 'param_count': 0, 'total_price': 0, 'epochs_completed': 0, 'events': [{'object': 'fine-tune-event', 'created_at': '2023-09-22T20:37:12.007Z', 'level': '', 'message': 'Fine tune request created', 'type': 'JOB_PENDING', 'param_count': 0, 'token_count': 0, 'checkpoint_path': '', 'model_path': '', 'training_offset': 0, 'hash': ''}], 'queue_depth': 0, 'wandb_key': '', 'wandb_project_name': '', 'enable_checkpoints': False}```

In [ ]:
# run this from time to time to check on the status of your job
print(together.Finetune.retrieve(fine_tune_id=fine_tune_id)) # retrieves information on finetune event
print("-"*50)
print(together.Finetune.get_job_status(fine_tune_id=fine_tune_id)) # pending, running, completed
print(together.Finetune.is_final_model_available(fine_tune_id=fine_tune_id)) # True, False
print(together.Finetune.get_checkpoints(fine_tune_id=fine_tune_id)) # list of checkpoints

{'training_file': 'file-b74a81d1-fce5-4868-ae3a-51a2c7fcf5c0', 'model_output_name': 'carson/llama-2-7b-chat-law-2023-09-23-05-25-27', 'model_output_path': 's3://together-dev/finetune/64c4302a5cb247a0c80a3ddb/carson/llama-2-7b-chat-law-2023-09-23-05-25-27/ft-e57a3227-acca-47e2-9a1b-4bcee8945e09', 'Suffix': 'law', 'model': 'togethercomputer/llama-2-7b-chat', 'n_epochs': 2, 'n_checkpoints': 1, 'batch_size': 4, 'learning_rate': 5e-05, 'user_id': '64c4302a5cb247a0c80a3ddb', 'staring_epoch': 0, 'training_offset': 0, 'checkspoint_path': '', 'random_seed': '', 'created_at': '2023-09-23T05:25:27.755Z', 'updated_at': '2023-09-23T05:25:31.527Z', 'status': 'running', 'owner_address': '0xef5286fc0a1ac5bc4d4221cf3d51f1d97c45eaf7', 'id': 'ft-e57a3227-acca-47e2-9a1b-4bcee8945e09', 'job_id': '1515', 'token_count': 0, 'param_count': 0, 'total_price': 0, 'epochs_completed': 0, 'events': [{'object': 'fine-tune-event', 'created_at': '2023-09-23T05:25:27.755Z', 'level': '', 'message': 'Fine tune request cre

Monitor your training/finetuning progress using weights and biases. Below we verify that our model is learning and our training loss is decreasing.

<center><img src="https://raw.githubusercontent.com/clam004/together-examples/main/files/wandb.jpg" height=300 width=600> </center>


In [ ]:
print(together.Finetune.get_job_status(fine_tune_id=fine_tune_id)) # pending, running, completed

running


get_job_status will transition from `pending`, `queued`, `running`, to `complete`.

when the job is finished, you should see:
```
{'training_file': 'file-69649a68-6a36-41ad-8420-1750e99c26a7', 'model_output_name': 'carson/llama-2-7b-chat-law-2023-09-22-20-37-12', 'model_output_path': 's3://together-dev/finetune/64c4302a5cb247a0c80a3ddb/carson/llama-2-7b-chat-law-2023-09-22-20-37-12/ft-2aaecf7b-ff6f-4341-813a-45d84ae2b1bf-2023-09-22-13-47-25', 'Suffix': 'law', 'model': 'togethercomputer/llama-2-7b-chat', 'n_epochs': 2, 'n_checkpoints': 1, 'batch_size': 4, 'learning_rate': 0.0001, 'user_id': '64c4302a5cb247a0c80a3ddb', 'staring_epoch': 0, 'training_offset': 0, 'checkspoint_path': '', 'random_seed': '', 'created_at': '2023-09-22T20:37:12.007Z', 'updated_at': '2023-09-22T20:52:52.675Z', 'status': 'completed', 'owner_address': '0xef5286fc0a1ac5bc4d4221cf3d51f1d97c45eaf7', 'id': 'ft-2aaecf7b-ff6f-4341-813a-45d84ae2b1bf', 'job_id': '1509', 'token_count': 1676524, 'param_count': 6738415616, 'total_price': 5000000000, 'epochs_completed': 2, 'events': [{'object': 'fine-tune-event', 'created_at': '2023-09-22T20:37:12.007Z .....

completed

True
[]
```

The name of your finetuned model will show up in your list of models, but before you can start using it, you need to start it and it needs to finish deploying.

You can also find the name of your new model, start it and stop it, at https://api.together.xyz/playground

under `Models` > `My Model Instances`

<center><img src="https://raw.githubusercontent.com/clam004/together-examples/main/files/mymodels.jpg" height=300 width=600></center>

In [ ]:
# replace this name with the name of your newly finetuned model
new_model_name = 'carson/ft-2aaecf7b-ff6f-4341-813a-45d84ae2b1bf-2023-09-22-13-47-25'

model_list = together.Models.list()

print(f"{len(model_list)} models available")

available_model_names = [model_dict['name'] for model_dict in model_list]

new_model_name in available_model_names

In [ ]:
# deploy your newly finetuned model
together.Models.start(new_model_name)

In [ ]:
# check if your model is finished deploying, if this returns {"ready": true}, you model is ready for inference
together.Models.ready(new_model_name)

In [ ]:
# use the inference API to generate text / create completion / chat
print(new_model_name)

output = together.Complete.create(
  prompt = test_chat_prompt,
  model = new_model_name,
  max_tokens = 256,
  temperature = 0.6,
  top_k = 90,
  top_p = 0.8,
  repetition_penalty = 1.1,
  stop = ['</s>']
)

# print generated text
print(output['prompt'][0]+" -> "+output['output']['choices'][0]['text'])

#### Example input separated from generated output by the ` -> ` sequence:


`<s>[INST] <<SYS>> you are a helpful legal assistant <</SYS>> Analyze and explain the legal reasoning behind the judgment in the given case.\n\nCentral Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly & Anr., 1986 AIR 1571, 1986 SCR (2) 278 [/INST]`

 ` -> `

 `The Supreme Court's decision in Central Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly & Anr. was based on several key legal principles and reasoning. Firstly, the court held that Article 14 of the Indian Constitution, which guarantees the right to equality, can apply to contractual terms between State entities and private parties if found arbitrary. This means that even though the employees were not government servants under the definition provided under Article 12, their service conditions could still be subject to constitutional scrutiny. Secondly, the court emphasized that Article 14 does not require proof of malice or mala fides for a termination to be deemed arbitrary. Instead, it is enough to show that the termination violates the principle of reasonableness and non-arbitrariness. Thirdly, the court rejected the doctrine of 'State instrumentality' as a valid defense against writ jurisdiction, stating that an entity will be considered 'State' only if there is deep and pervasive governmental control in its functioning and financing. Lastly, the court clarified that the right to livelihood under Article 21 is affected by arbitrary termination of employment. Therefore, .....`
  `</s>`

##### *the stop token is `</s>`*

In [ ]:
# stop your model and you will no longer be paying for it
together.Models.stop(new_model_name)